<a href="https://colab.research.google.com/github/NnekaAsuzu/fine-tuning-on-any-question-answering-dataset-from-HuggingFace/blob/main/Finetuning_on_Hugging_Face_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 **Project**: Perform fine-tuning on any question-answering dataset from HuggingFace and save the model. Use the saved model to build a Gradio interface. The interface should display a context window as the first input, a text field for the user's question as the second input, and the model's response as the output. This interface will allow users to input a context and a question to receive the model's answer.




#####Install the Transformers, Datasets, and Evaluate libraries to run this notebook.

In [1]:
!pip install datasets evaluate transformers[sentencepiece]
!pip install accelerate
!apt install git-lfs
!pip install datasets
!pip install transformers datasets torch gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 6.7 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 48.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 15.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.7/310.7 kB 42.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 13.0 MB/s eta 0:00:00
   

##setup git, adapt your email and name in the following cell

In [2]:
!git config --global user.email "nnasuzu@gmail.com"
!git config --global user.name "NnekaAsuzu"

##log in to the Hugging Face Hub

In [3]:
from huggingface_hub import notebook_login

notebook_login()

In [4]:
#Import  libraries
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, Trainer, TrainingArguments,DataCollatorWithPadding
import torch
from transformers import default_data_collator
from torch.utils.data import DataLoader
import gradio as gr

In [5]:
# Download Load the dataset Stanford Question Answering Dataset (SQuAD)

from datasets import load_dataset

dataset = load_dataset("Squad")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [6]:
dataset #viewdatasets

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

In [7]:
dataset['train'][0] #to look at the data

{'id': '5733be284776f41900661182',
 'title': 'University_of_Notre_Dame',
 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
 'answers': {'text': ['Saint Bernadette Soubirous'], 'answer_start': [515]}}

In [8]:
 # Load the tokenizer and model(in other to tokenize any sentences, we have to define the tockenizer first and load)
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModelForQuestionAnswering.from_pretrained("bert-base-uncased")
# Load a pretrained model
pretrained_model = AutoModelForQuestionAnswering.from_pretrained("bert-base-uncased")



tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
#load checkpoint
checkpoint ='bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(checkpoint) #load tokenizer from the checkpoint

In [10]:
dataset['train']['context'][0] #to view the context

'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.'

In [11]:
# Tokenize the dataset
def tokenize_function(example):
    return tokenizer(example["question"], example["context"], truncation=True)
    #map is used to apply tokenizer function in the whole dataset
#batch is used to apply function in batches and iterate over the dataset much faster

tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/87599 [00:00<?, ? examples/s]

Map:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [12]:
# compare the orginal to the tockenized dataset ( more inputs are seen)
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

In [13]:
#To check the tokenized data
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 10570
    })
})

Training with a Custom Trainer Method: In this method, the custom trainer class is used to manage the training process.

In [14]:
# Define a custom training step
def compute_loss(model, inputs):
    outputs = model(**inputs)
    start_logits = outputs.start_logits
    end_logits = outputs.end_logits
    start_positions = inputs.get("start_positions")
    end_positions = inputs.get("end_positions")
    loss = None
    if start_positions is not None and end_positions is not None:
        loss_fct = torch.nn.CrossEntropyLoss()
        start_loss = loss_fct(start_logits, start_positions)
        end_loss = loss_fct(end_logits, end_positions)
        loss = (start_loss + end_loss) / 2
    return loss if loss is not None else torch.tensor(0.0, device=model.device, requires_grad=True)

In [15]:
 #Define the trainer with the overridden compute_loss method
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs):
        return compute_loss(model, inputs)


In [16]:
# Define training arguments with modified logging steps
training_args = TrainingArguments(
    output_dir="./checkpoints",  # Directory to save checkpoints
    save_strategy="steps",        # Save checkpoints every few steps
    save_steps=500,               # Save a checkpoint every 500 steps
    logging_dir="./logs",         # Directory to save logs
    logging_steps=2000,           # Log metrics every 2000 steps
    overwrite_output_dir=True,    # Overwrite the output directory if it exists
    # Add other training arguments as needed
)

In [17]:
# Create an instance of the custom trainer with the modified training arguments
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=DataCollatorWithPadding(tokenizer),
)

In [18]:
# Train the model
trainer.train()

Step,Training Loss
2000,0.000000
4000,0.000000
6000,0.000000
8000,0.000000
10000,0.000000
12000,0.000000
14000,0.000000
16000,0.000000
18000,0.000000
20000,0.000000


TrainOutput(global_step=32850, training_loss=0.0, metrics={'train_runtime': 5127.3535, 'train_samples_per_second': 51.254, 'train_steps_per_second': 6.407, 'total_flos': 3.708946805189746e+16, 'train_loss': 0.0, 'epoch': 3.0})

In [19]:
# Save the fine-tuned model
model.save_pretrained("fine_tuned_squad_model")
tokenizer.save_pretrained("fine_tuned_squad_model")


('fine_tuned_squad_model/tokenizer_config.json',
 'fine_tuned_squad_model/special_tokens_map.json',
 'fine_tuned_squad_model/vocab.txt',
 'fine_tuned_squad_model/added_tokens.json',
 'fine_tuned_squad_model/tokenizer.json')

In [20]:
# Load the saved model and tokenizer
model_path = "fine_tuned_squad_model"
model = AutoModelForQuestionAnswering.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

In [21]:
#Generate sample responses from the model
# Define a function to get the model's response/predict the answer
def answer_question(context, question):
    inputs = tokenizer(question, context, return_tensors="pt")
    input_ids = inputs["input_ids"].tolist()[0]

    outputs = model(**inputs)
    answer_start = torch.argmax(outputs.start_logits)
    answer_end = torch.argmax(outputs.end_logits) + 1
    answer = tokenizer.decode(input_ids[answer_start:answer_end])

    return answer

# Context
context = """
AI is revolutionizing industries globally, impacting job markets.
"""

# Questions
question = [
    "How is AI revolutionizing industries globally?"
]

# Answer each question
for i, question in enumerate(question, 1):
    answer = answer_question(context, question)
    print(f"Question {i}: {question}\nAnswer: {answer}\n")


Question 1: How is AI revolutionizing industries globally?
Answer: revolutionizing industries globally, impacting



In [22]:
# Create a Gradio interface
context_input = gr.Textbox(lines=10, label="Context")
question_input = gr.Textbox(lines=2, label="Question")
output_text = gr.Textbox(label="Answer")

# Launch the interface
iface = gr.Interface(fn=answer_question, inputs=[context_input, question_input], outputs=output_text)
iface.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://b1e7694b41839477b9.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
